In [2]:
#Generate a code to develop a model using Reinforcement_Learning
'''import gym
import random
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n
model = Sequential()
model.add(Flatten(input_shape=(1,states)))
model.add(Dense(24, activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(actions, activation='linear'))
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model
model = build_model(states, actions)
model.summary()
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn
    dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))
_ = dqn.test(env, nb_episodes=15, visualize=True)
'''

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


ModuleNotFoundError: No module named 'rl'

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from google.colab import files
uploaded = files.upload()

Saving aapl_us_d.csv to aapl_us_d (2).csv


In [20]:
'''
# Load data
data = pd.read_csv(next(iter(uploaded)))

# Ensure 'SMA_5' and 'SMA_10' columns are calculated if not present
if 'SMA_5' not in data.columns:
    data['SMA_5'] = data['Close'].rolling(window=5).mean()
if 'SMA_10' not in data.columns:
    data['SMA_10'] = data['Close'].rolling(window=10).mean()

# Drop rows with NaN values resulting from rolling calculations
data = data.dropna().reset_index(drop=True)
data['Price_Change'] = data['Close'].diff()

# Define the Q-learning setup
actions = ['buy', 'hold', 'sell']
q_table = {}

# Hyperparameters for Q-learning
alpha = 0.1
gamma = 0.95
epsilon = 0.1

# Initial capital for trading
capital = 1000
shares_held = 0
portfolio_value = []

# Function to get current state based on feature values
def get_state(row):
    return (round(row['Price_Change'], 4), round(row['SMA_5'], 2), round(row['SMA_10'], 2))

# Epsilon-greedy action selection
def choose_action(state):
    if np.random.rand() < epsilon or state not in q_table:
        return np.random.choice(actions)
    return max(q_table[state], key=q_table[state].get)

# Q-value update
def update_q_table(state, action, reward, next_state):
    if state not in q_table:
        q_table[state] = {a: 0 for a in actions}
    if next_state not in q_table:
        q_table[next_state] = {a: 0 for a in actions}

    best_future_q = max(q_table[next_state].values())
    q_table[state][action] += alpha * (reward + gamma * best_future_q - q_table[state][action])

# Simulate trading with Q-learning
for i in range(len(data) - 1):
    state = get_state(data.iloc[i])
    action = choose_action(state)
    next_state = get_state(data.iloc[i + 1])

    # Calculate reward based on action taken
    price_today = data.iloc[i]['Close']
    price_next = data.iloc[i + 1]['Close']

    if action == 'buy' and capital > price_today:
        shares_held += capital // price_today
        capital %= price_today
        reward = 0
    elif action == 'sell' and shares_held > 0:
        capital += shares_held * price_today
        reward = (price_today - data.iloc[i - 1]['Close']) * shares_held
        shares_held = 0
    else:
        reward = 0

    # Update Q-table and record portfolio value
    update_q_table(state, action, reward, next_state)
    portfolio_value.append(capital + shares_held * price_next)

# Final portfolio value
final_portfolio_value = portfolio_value[-1]

# Results summary
q_table_summary = {
    'Initial Capital': 10,
    'Final Capital': capital + shares_held * data.iloc[-1]['Close'],
    'Final Portfolio Value': final_portfolio_value,
    'Total Profit/Loss': final_portfolio_value - 1000
}
q_table_summary
'''

{'Initial Capital': 10,
 'Final Capital': 1291280.7202226005,
 'Final Portfolio Value': 1291280.7202226005,
 'Total Profit/Loss': 1290280.7202226005}

In [28]:
import numpy as np
import pandas as pd

# Set a random seed for reproducibility in training, but comment this out for varied results
# np.random.seed(42)

# Load data
data = pd.read_csv(next(iter(uploaded)))

# Ensure 'SMA_5' and 'SMA_10' columns are calculated if not present
if 'SMA_5' not in data.columns:
    data['SMA_5'] = data['Close'].rolling(window=5).mean()
if 'SMA_10' not in data.columns:
    data['SMA_10'] = data['Close'].rolling(window=10).mean()

# Calculate 'Price_Change' if it is not present
if 'Price_Change' not in data.columns:
    data['Price_Change'] = data['Close'].diff()

# Drop rows with NaN values resulting from calculations
data = data.dropna().reset_index(drop=True)

# Define the Q-learning setup
actions = ['buy', 'hold', 'sell']
q_table = {}

# Hyperparameters for Q-learning
alpha = 0.1
gamma = 0.95
epsilon = 0.1  # Exploration enabled in training

# Initial capital for trading
capital = 1000
shares_held = 0
portfolio_value = []

# Function to get current state based on feature values
def get_state(row):
    return (round(row['Price_Change'], 4), round(row['SMA_5'], 2), round(row['SMA_10'], 2))

# Epsilon-greedy action selection
def choose_action(state):
    if np.random.rand() < epsilon or state not in q_table:
        return np.random.choice(actions)
    return max(q_table[state], key=q_table[state].get)

# Q-value update
def update_q_table(state, action, reward, next_state):
    if state not in q_table:
        q_table[state] = {a: 0 for a in actions}
    if next_state not in q_table:
        q_table[next_state] = {a: 0 for a in actions}

    best_future_q = max(q_table[next_state].values())
    q_table[state][action] += alpha * (reward + gamma * best_future_q - q_table[state][action])

# Training Phase
for i in range(len(data) - 1):
    state = get_state(data.iloc[i])
    action = choose_action(state)
    next_state = get_state(data.iloc[i + 1])

    price_today = data.iloc[i]['Close']
    price_next = data.iloc[i + 1]['Close']

    if action == 'buy' and capital > price_today:
        shares_held += capital // price_today
        capital %= price_today
        reward = 0
    elif action == 'sell' and shares_held > 0:
        capital += shares_held * price_today
        reward = (price_today - data.iloc[i - 1]['Close']) * shares_held
        shares_held = 0
    else:
        reward = 0

    update_q_table(state, action, reward, next_state)
    portfolio_value.append(capital + shares_held * price_next)

# Evaluation Phase with epsilon set to allow exploration
epsilon = 0.2  # Non-zero epsilon for varied results
capital = 10
shares_held = 0
portfolio_value.clear()

for i in range(len(data) - 1):
    state = get_state(data.iloc[i])
    action = choose_action(state)
    price_today = data.iloc[i]['Close']
    price_next = data.iloc[i + 1]['Close']

    if action == 'buy' and capital > price_today:
        shares_held += capital // price_today
        capital %= price_today
    elif action == 'sell' and shares_held > 0:
        capital += shares_held * price_today
        shares_held = 0

    portfolio_value.append(capital + shares_held * price_next)

# Final portfolio value after evaluation
final_portfolio_value = portfolio_value[-1]
q_table_summary = {
    'Initial Capital': 10,
    'Final Capital': capital + shares_held * data.iloc[-1]['Close'],
    'Final Portfolio Value': final_portfolio_value,
    'Total Profit/Loss': final_portfolio_value - 10
}
q_table_summary


{'Initial Capital': 10,
 'Final Capital': 7944.854354199997,
 'Final Portfolio Value': 7944.854354199997,
 'Total Profit/Loss': 7934.854354199997}